In [1]:
import os
import pandas as pd
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score, matthews_corrcoef, auc, plot_precision_recall_curve,precision_recall_curve
from statistics import median
from glob import glob
import os, pickle
import numpy as np
import os
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)


libs = [ 'rf', 'xgb', 'lr']
names = ['TAILED', 'NOTNORMAL', 'CONSTFEA', 'PKG', 'SCHEMA', 'CORREDUN', 'MISLABEL2', 'CLEAN', 'CLSOVERLAP', 'MISLABCLSOV']
files = pd.read_csv('../oracle.csv')['file'].to_list()
result_dir = 'Results_23/'

k = os.listdir(result_dir)
print(len(k))

len(libs) * len(names) * len(files)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
900


900

### starting algo

In [ ]:
# results: All other metrics are low even though the AUC is high. 
# need to check other papers which use this dataset, on what they reported? 

In [3]:
# testing code

lib = 'rf'
file = 'activemq-5.2.0'
name = 'PKG'

def get_auprauc(x):
    precision, recall, thresholds = precision_recall_curve(x.y_test, x.probs)
    auc_precision_recall = auc(recall, precision)
    return auc_precision_recall

res_pkl = result_dir + file + '_' + name + '_' + lib + '.pkl'    
df_type = pd.read_pickle(res_pkl)
df_type['recall'] = df_type.apply(lambda x: recall_score(x.y_test, x.y_pred), axis=1)
df_type['precision'] = df_type.apply(lambda x: precision_score(x.y_test, x.y_pred), axis=1)
df_type['f1'] = df_type.apply(lambda x: f1_score(x.y_test, x.y_pred), axis=1)
df_type['mcc'] = df_type.apply(lambda x: matthews_corrcoef(x.y_test, x.y_pred), axis=1)
df_type['auprauc'] = df_type.apply(get_auprauc, axis=1)

df_type.head(2)

auc                                               imps lib  \
0  0.818806                                                 {}  rf   
0  0.797265  {'CountClassCoupled': 0.03, 'CountPath_Min': 0...  rf   

                                              y_pred  \
0  [False, False, False, False, False, False, Fal...   
0  [True, False, False, False, False, False, Fals...   

                                              y_test  \
0  0      0
1      0
2      0
3      0
4      0
 ...   
0  0      1
1      0
2      0
3      0
4      0
 ...   

                                               probs  \
0  [0.30342806705876363, 0.0, 0.0, 0.029022187561...   
0  [0.90309362436435, 0.009714230960867732, 0.0, ...   

                                               model  \
0  (DecisionTreeClassifier(criterion='entropy', m...   
0  (DecisionTreeClassifier(criterion='entropy', m...   

                                           filename    recall  precision  \
0  Data_oneButall/BootStrapData_23/activemq-5.2.0_0  0.568182   0.446429   
0  Data_oneButall/BootStrapData_23/activemq-5.2.0_1  0.568182   0.384615   

         f1       mcc   auprauc  
0  0.500000  0.435386  0.511456  
0  0.458716  0.388432  0.502734

In [14]:
# # the return of the model i.e., model.predict and the round(model.predict_proba) are the same!, as checked below!

# row = df_type.iloc[0]
# print((row.y_pred)[:8])
# print((row.probs)[:8])
# print((row.y_test)[:8])
# k = row.probs

# y_pred = row.y_pred.astype(int)
# tmp = list(map(lambda x: round(x), k))

# all(map(lambda x, y: x == y, y_pred, tmp))

# Getting only AUC df

In [4]:
def get_metrics(file, name, lib):
    
    def get_auprauc(x):
        precision, recall, thresholds = precision_recall_curve(x.y_test, x.probs)
        auc_precision_recall = auc(recall, precision)
        return auc_precision_recall

    res_pkl = result_dir + file + '_' + name + '_' + lib + '.pkl'    
    
    df_type = pd.read_pickle(res_pkl)
    df_type['recall'] = df_type.apply(lambda x: recall_score(x.y_test, x.y_pred), axis=1)
    df_type['precision'] = df_type.apply(lambda x: precision_score(x.y_test, x.y_pred), axis=1)
    df_type['f1'] = df_type.apply(lambda x: f1_score(x.y_test, x.y_pred), axis=1)
    df_type['mcc'] = df_type.apply(lambda x: matthews_corrcoef(x.y_test, x.y_pred), axis=1)
    df_type['auprauc'] = df_type.apply(get_auprauc, axis=1)
    
    return df_type[['auc', 'lib', 'recall', 'precision', 'f1', 'mcc', 'auprauc']]


In [ ]:
def getAuc(file):
    master_df = pd.DataFrame()
    for lib in libs:
        for name in names:
            res_df = get_metrics(file, name, lib)
            res_df['name'] = name
            res_df['file'] = file
            master_df = pd.concat([master_df,res_df])
            
    return master_df

auc_df = pd.concat(map(getAuc, files[:2]))


from  multiprocessing import Pool
p = Pool()
metrics_df = pd.concat(p.map(getAuc, files))
p.terminate()
p.close()

In [7]:
metrics_df.to_pickle('metrics_rf_xgb_lr_23.pkl')

In [2]:
metrics_df = pd.read_pickle('metrics_rf_xgb_lr_23.pkl')

# final checks!
print(len(metrics_df))
len(metrics_df) == len(libs) * len(names) * len(files) * 10 # for 10 iterations

9000


True

In [3]:
metrics_df.head()

auc lib    recall  precision        f1       mcc   auprauc    name  \
0  0.845805  rf  0.694915   0.455556  0.550336  0.460931  0.570850  TAILED   
0  0.851002  rf  0.627119   0.578125  0.601626  0.524744  0.657573  TAILED   
0  0.824805  rf  0.305085   0.620690  0.409091  0.368814  0.583676  TAILED   
0  0.842101  rf  0.457627   0.562500  0.504673  0.426827  0.593537  TAILED   
0  0.899211  rf  0.779661   0.469388  0.585987  0.510391  0.698165  TAILED   

             file  
0  activemq-5.0.0  
0  activemq-5.0.0  
0  activemq-5.0.0  
0  activemq-5.0.0  
0  activemq-5.0.0

In [5]:
# make in range 0.5 to 1

def cleanRange(x):
    if x >=0.5:
        return x
    else:
        return 1-x
    
metrics_df['precision'] = metrics_df['precision'].apply(cleanRange)
metrics_df['recall'] = metrics_df['recall'].apply(cleanRange)
metrics_df['f1'] = metrics_df['f1'].apply(cleanRange)
metrics_df['mcc'] = metrics_df['mcc'].apply(cleanRange)
metrics_df['auprauc'] = metrics_df['auprauc'].apply(cleanRange)

metrics_df.head()

auc lib    recall  precision        f1       mcc   auprauc    name  \
0  0.845805  rf  0.694915   0.544444  0.550336  0.539069  0.570850  TAILED   
0  0.851002  rf  0.627119   0.578125  0.601626  0.524744  0.657573  TAILED   
0  0.824805  rf  0.694915   0.620690  0.590909  0.631186  0.583676  TAILED   
0  0.842101  rf  0.542373   0.562500  0.504673  0.573173  0.593537  TAILED   
0  0.899211  rf  0.779661   0.530612  0.585987  0.510391  0.698165  TAILED   

             file  
0  activemq-5.0.0  
0  activemq-5.0.0  
0  activemq-5.0.0  
0  activemq-5.0.0  
0  activemq-5.0.0

In [7]:
for l in libs:
    l_df = metrics_df[metrics_df.lib == l]
    print(l, l_df.auc.median(), l_df.recall.median(), l_df.precision.median(), l_df.f1.median(), l_df.mcc.median(), l_df.auprauc.median())

rf 0.7954110066179032 0.6538461538461539 0.7182153322620285 0.6666666666666667 0.7134131248318404 0.6331421484187076
xgb 0.7748633894447547 0.6666666666666666 0.7313432835820896 0.6900656814449918 0.7382279318870424 0.6446416688414314
lr 0.7591892113433223 0.6585365853658537 0.7428571428571429 0.6715086117451754 0.7322278958045729 0.6158418962654549
